# 03-filter_nodes_edges.ipynb
- Remove edges with nodes with no umls type or label
- Remove the following predicates: ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 
'same_as','OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES','AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES']

In [1]:
import os
import pickle
%pylab
%matplotlib inline
import pandas as pd
from semmeddb_biolink_environment import *

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
edges = pd.read_csv(EDGES4_TSV, sep='\t', index_col=0)
edges.head()

/Users/richard/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SUBJECT_CUI,PREDICATE,OBJECT_CUI,PMID,NEG
766,C1412045,ADMINISTERED_TO,C0007634,24096582,False
767,C1412045,ADMINISTERED_TO,C0030705,22577025,False
768,C1412045,AFFECTS,C0005935,29798367,False
769,C1412045,AFFECTS,C0020291,6298464,False
770,C1412045,AFFECTS,C0028754,19789049,False


In [3]:
nodes = pd.read_csv(NODES_BLM_TSV, sep='\t', index_col=0)
nodes.head()

,ID,LABEL,umls_type,umls_type_label,blm_category
0,C0007952,Personality Character,T041,Mental Process,Behavior
1,C3574797,Pbunavirus,T005,Virus,OrganismTaxon
2,C0948102,Salivary gland adenoma,T191,Neoplastic Process,Disease
3,C0210064,quinotolast,T121|T109,Pharmacologic Substance|Organic Chemical,MolecularEntity
4,C1008523,Silene viscosa,T002,Plant,OrganismTaxon


In [4]:
print(len(nodes))
print(len(edges))

310264
26370979


In [5]:
# remove edges with no nodes
print(len(edges))
edges = edges[edges.SUBJECT_CUI.isin(nodes.ID) & edges.OBJECT_CUI.isin(nodes.ID)]
print(len(edges))

26370979
24696075


In [6]:
vc = edges.PREDICATE.value_counts()
vc

LOCATION_OF         3308116
INTERACTS_WITH      2536607
AFFECTS             2355724
COEXISTS_WITH       2113139
TREATS              1517832
STIMULATES          1437115
PART_OF             1371158
INHIBITS            1219056
ASSOCIATED_WITH     1146716
PROCESS_OF          1096299
CAUSES              1066883
AUGMENTS             694213
DISRUPTS             658150
compared_with        549620
PRODUCES             540309
USES                 525300
PREDISPOSES          378655
ISA                  372949
DIAGNOSES            318167
ADMINISTERED_TO      270170
PREVENTS             250853
MEASURES             237404
PRECEDES             169184
higher_than          154528
METHOD_OF            139295
OCCURS_IN             61291
CONVERTS_TO           53282
COMPLICATES           47268
MANIFESTATION_OF      45009
same_as               32353
lower_than            28258
MEASUREMENT_OF         1166
PREP                      4
1532                      1
NOM                       1
Name: PREDICATE, dty

In [7]:
def drop_edges(df, drop_edges):
    idx = df.query('PREDICATE in @drop_edges').index
    df.drop(idx, inplace=True)

In [8]:
# remove edges we don't need
print(len(edges))
remove_edges = ['compared_with', 'higher_than', 'lower_than', 'different_from', 'different_than', 'same_as',
               'OCCURS_IN', 'PROCESS_OF', 'DIAGNOSES', 'METHOD_OF', 'USES',
               'AUGMENTS', 'ADMINISTERED_TO', 'COMPLICATES', '1532', 'PREP', 'NOM']
drop_edges(edges, remove_edges)
print(len(edges))

24696075
20779307


In [9]:
# remove nodes with no edges
print(len(nodes))
nodes = nodes[nodes.ID.isin(set(list(edges.SUBJECT_CUI) + list(edges.OBJECT_CUI)))]
print(len(nodes))

310264
291339


In [10]:
vc = edges.PREDICATE.value_counts()
vc

LOCATION_OF         3308116
INTERACTS_WITH      2536607
AFFECTS             2355724
COEXISTS_WITH       2113139
TREATS              1517832
STIMULATES          1437115
PART_OF             1371158
INHIBITS            1219056
ASSOCIATED_WITH     1146716
CAUSES              1066883
DISRUPTS             658150
PRODUCES             540309
PREDISPOSES          378655
ISA                  372949
PREVENTS             250853
MEASURES             237404
PRECEDES             169184
CONVERTS_TO           53282
MANIFESTATION_OF      45009
MEASUREMENT_OF         1166
Name: PREDICATE, dtype: int64

In [12]:
edges.to_csv(EDGES_FILTERED_TSV, sep='\t', index=False)
nodes.to_csv(NODES_FILTERED_TSV, sep='\t', index=False)